In [ ]:
!pip install --upgrade diffusers[torch]
!pip install transformers
!pip install matplotlib
!pip install opencv-python
!pip install -U peft
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
model_name = "ViT-B/32"
model, preprocess = clip.load(model_name)

In [ ]:
import torch
import torch.nn.functional as F
from torch import optim
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np 
import clip

def inverse_image_to_text_embedding(model, processor, image, text_embedding, num_steps=500, lr=1e-2):
    # Preprocess image
    image_input = torch.tensor(np.stack([preprocess(image)])).cuda()
    
    # Get image features
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
    
    # Project image features into the joint embedding space
    I_e = F.normalize(image_features, dim=0)  # Shape: [batch_size, d_e]
    print(image_features.shape, I_e.shape)
    
    # Initialize text embedding tensor
    text_embedding_dim = 512
    seq_len = 77
    text_embedding = torch.randn((1, seq_len, text_embedding_dim), requires_grad=True, device="cuda")
    
    # Define the optimizer
    optimizer = optim.Adam([text_embedding], lr=lr)
    
    for step in range(500):
        optimizer.zero_grad()
        
        # Normalize the text embedding
        T_e = F.normalize(text_embedding, dim=1)
        print(text_embedding.shape)
        # Compute cosine similarity
        similarity = torch.matmul(image_features, T_e.mean(dim=1).T)
        print(similarity.shape, image_features.shape, T_e.mean(dim=1).T.shape)
        # Define the loss (negative cosine similarity)
        loss = -similarity.mean()
    
        # Backpropagation
        loss.backward()
        optimizer.step()
        print(f'Step [{step}/{num_steps}], Loss: {loss.item()}')
            
    return text_embedding			

# Definitions
model_name = "ViT-B/32"
image_name = "1.png"

num_steps = 500
lr = 1e-2

image = Image.open(image_name)
image.show()
output_tensor = inverse_image_to_text_embedding(model, processor, image, num_steps, lr)

torch.save(output_tensor, f"emb_{image_name}.pt")
print(f"Optimized text embeddings saved to 'optimized_text_embedding_{image_name}.pt'")

